In [1]:
%load_ext autoreload 
%autoreload 2

import logging

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.v2 as transforms
from torchsummary import summary

from medseg.datasets import PH2Dataset
from medseg.models import EncDec
from medseg.training import train
from medseg.metrics import dice_coefficient, iou_loss, accuracy, sensitivity, specificity, bce_loss

logging.basicConfig(level=logging.INFO, format='%(name)s: %(message)s')

In [2]:
#logging.getLogger().setLevel(logging.DEBUG)

In [3]:
torch.cuda.set_device(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
trainset = PH2Dataset('train')
testset = PH2Dataset('test')
valset = PH2Dataset('validation')

batch_size = 6
train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=3,
)
test_loader = torch.utils.data.DataLoader(
    testset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=3,
)

## Training

In [5]:
model = EncDec().to(device)
summary(model, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 254, 254]           1,792
         MaxPool2d-2         [-1, 64, 127, 127]               0
            Conv2d-3         [-1, 64, 125, 125]          36,928
         MaxPool2d-4           [-1, 64, 62, 62]               0
            Conv2d-5           [-1, 64, 60, 60]          36,928
         MaxPool2d-6           [-1, 64, 30, 30]               0
            Conv2d-7           [-1, 64, 28, 28]          36,928
         MaxPool2d-8           [-1, 64, 14, 14]               0
            Conv2d-9           [-1, 64, 12, 12]          36,928
         Upsample-10           [-1, 64, 16, 16]               0
           Conv2d-11           [-1, 64, 14, 14]          36,928
         Upsample-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 30, 30]          36,928
         Upsample-14           [-1, 64,

In [6]:
epochs = 3
dice_loss_val_list, iou_loss_val_list, acc_val_list, sens_val_list, spec_val_list = train(model, torch.optim.Adam(model.parameters()), bce_loss, epochs, train_loader, test_loader, device)

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/mlut/.conda/envs/bipolardata/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/mlut/.conda/envs/bipolardata/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/mlut/.conda/envs/bipolardata/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/mlut/IDLCV/PH2_Dataset_images/02516_intro_cnn_w2/src/medseg/datasets.py", line 76, in __getitem__
    image = Image.open(image_path)
  File "/home/mlut/.conda/envs/bipolardata/lib/python3.10/site-packages/PIL/Image.py", line 3218, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such 